In [ ]:
# def mixing_weights_3(alpha):
#     w_for_first = [0.5, 0.3, 0.2]
#     w_for_second = [0.4,0.5,0.1]
#     w_for_third = [0.15 ,0.2,0.4]
#     return w_for_first, w_for_second, w_for_third

In [ ]:
# from disentangle.core.tiff_reader import load_tiff
# data = load_tiff('/group/jug/ashesh/data/microscopy/OptiMEM100x014_medium_spatially_large.tif')

In [ ]:
# _,ax = plt.subplots(figsize=(12,4),ncols=3)
# ax[0].imshow(act_tub[...,0])
# ax[1].imshow(act_tub[...,1])
# ax[2].imshow(act_tub[...,2])

In [ ]:
# import matplotlib.pyplot as plt
# plt.imshow(data[0,::4,::4,3])

In [ ]:
# from disentangle.core.tiff_reader import save_tiff
# import numpy as np
# act_tub  = data[0,...,[0,2,3]]
# act_tub = np.moveaxis(act_tub, 0, -1)
# act_tub.shape
# # save_tiff('picasso_input_pavia_nuc_ac_tub.tif', act_tub)

In [ ]:
# act_tub.shape
# output = np.zeros_like(act_tub)
# w0,w1,w2 = mixing_weights_3(0.15)
# print(w0)
# print(w1)
# print(w2)
# output[...,0] = act_tub[...,0] * w0[0] + act_tub[...,1] * w0[1] + act_tub[...,2] * w0[2]/np.sum(w0)
# output[...,1] = (act_tub[...,0] * w1[0] + act_tub[...,1] * w1[0] + act_tub[...,2] * w1[2])/np.sum(w1)
# output[...,2] = (act_tub[...,0] * w2[0] + act_tub[...,1] * w2[1] + act_tub[...,2] * w2[2])/np.sum(w2)
# # for i in range

In [ ]:
output.std()

In [ ]:
# noise = np.random.normal(scale=200, size = output.shape)
# noisy_output = output + noise
# noisy_output[noisy_output < 0] = 0
# noisy_output = noisy_output.astype(np.uint16)

In [ ]:
plt.imshow(noisy_output[...,2])

In [ ]:
# save_tiff('picasso_input_pavia_nuc_ac_tub.tif', noisy_output)

In [ ]:
break here

## Nicola

In [ ]:
def mixing_weights_logic(factor):
    """
    The brightest channel is factor times brighter than the second brightest channel
    x + x/factor + x/(factor*2) = 1
    """
    assert factor >= 1
    w_bright = factor/(factor + 1.5)
    w_mid = 1/(factor + 1.5)
    w_dark = 0.5/(factor + 1.5)
    return {'bright': w_bright, 'mid': w_mid, 'dark': w_dark}
    # x + x/factor + x/(factor*2) = 1
    # x(factor + 1 + 0.5)/factor = 1
    # x = factor/(factor + 1.5)

def mixing_weights_for_three(factor):
    w_dict= mixing_weights_logic(factor)
    w_for_first = [w_dict['bright'], w_dict['mid'], w_dict['dark']]
    w_for_second = [w_dict['mid'], w_dict['bright'], w_dict['mid']]
    w_for_third = [w_dict['dark'], w_dict['mid'], w_dict['bright']]
    return w_for_first, w_for_second, w_for_third
    
def mixing_weights_for_three_asymmetric(factor, asymmetry_offset=0.05, reverse_factor_offset=0.05):
    w_dict= mixing_weights_logic(factor)
    w_for_first = [w_dict['bright'], w_dict['mid'], w_dict['dark']]
    w_for_second = [w_dict['mid']-asymmetry_offset, w_dict['bright'], w_dict['mid']+asymmetry_offset]
    
    w_dict = mixing_weights_logic(factor+reverse_factor_offset)
    w_for_third = [w_dict['dark'], w_dict['mid'], w_dict['bright']]
    return w_for_first, w_for_second, w_for_third
    

In [ ]:
a,b,c = mixing_weights_for_three(1.5)
print(a,b,c)

In [ ]:
# def mixing_weights_3(alpha):
#     w_for_first = [0.5, 0.3, 0.2]
#     w_for_second = [0.4,0.5,0.1]
#     w_for_third = [0.15 ,0.2,0.4]
#     return w_for_first, w_for_second, w_for_third

In [ ]:
from disentangle.data_loader.nikola_7D_rawdata_loader import get_raw_files_dict, load_one_fpath,NikolaChannelList, get_train_val_data
from disentangle.config_utils import load_config
import os
import numpy as np
from disentangle.core.data_split_type import DataSplitType

datadir = '/group/jug/ashesh/data/nikola_data/20240531/'
pkl_fpath = '/home/ashesh.ashesh/training/disentangle/2408/D25-M3-S0-L8/4'
config = load_config(pkl_fpath)
test_data= get_train_val_data(datadir, config.data, DataSplitType.Test, val_fraction=config.training.val_fraction, test_fraction=config.training.test_fraction)


# fnames = get_raw_files_dict()['500ms']
# fname = fnames[0]
# fnames
# data = load_one_fpath(os.path.join(datadir,fname),[NikolaChannelList.Ch_B, NikolaChannelList.Ch_C, NikolaChannelList.Ch_D,
                                                #    ])

In [ ]:
# nicola_image = data[0]
# nicola_image.shape
# for scale in np.arange(0, 1000, 100):
#     noise = np.random.normal(scale=scale, size = nicola_image.shape)
#     noisy_output = nicola_image + noise
#     print(scale, noisy_output[...,0].std()/noisy_output[:250,:250,0].std())

In [ ]:
# For 2ms exposure time, following is the std of just the noisy region and the full content std.
# 2.89 just noise. 
# 3.30 is full content std.

In [ ]:
from disentangle.core.tiff_reader import save_tiff
alpha = 2.5
save_to_file = False

for i,nicola_image in enumerate(test_data):
    output = np.zeros_like(nicola_image)
    w0,w1,w2 = mixing_weights_for_three(alpha)
    # w0,w1,w2 = mixing_weights_for_three_asymmetric(alpha)
    
    print(w0)
    output[...,0] = nicola_image[...,0] * w0[0] + nicola_image[...,1] * w0[1] + nicola_image[...,2] * w0[2]/np.sum(w0)
    output[...,1] = (nicola_image[...,0] * w1[0] + nicola_image[...,1] * w1[0] + nicola_image[...,2] * w1[2])/np.sum(w1)
    output[...,2] = (nicola_image[...,0] * w2[0] + nicola_image[...,1] * w2[1] + nicola_image[...,2] * w2[2])/np.sum(w2)

    noise = np.random.normal(loc=5000,scale=500, size = output.shape)
    # print(noise.std())
    noisy_output = output + noise
    print((noisy_output < 0).sum())
    noisy_output[noisy_output < 0] = 0
    noisy_output = noisy_output.astype(np.uint16)
    if save_to_file:
        save_tiff(f'picasso_input_nicola_BCD_alpha_{alpha}_{i}.tif', noisy_output)
    else:
        break

if save_to_file:
    save_tiff(f'picasso_clean_input_nicola_BCD_alpha_{alpha}_{i}.tif', output)

In [ ]:
# save_tiff(f'picasso_input_nicola_BCD_asymmetric_alpha_{alpha}_{i}.tif', noisy_output)

In [ ]:
import matplotlib.pyplot as plt
_,ax = plt.subplots(figsize=(18,6),ncols=3)
ax[0].imshow(noisy_output[:4000,:4000,0])
ax[1].imshow(noisy_output[:4000,:4000,1])
ax[2].imshow(noisy_output[:4000,:4000,2])